In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# To install SQLite
!apt-get update -q
!apt-get install -y sqlite3
!pip install pandas sqlite-utils

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,804 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [4,917 kB]
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving raw_orders.csv to raw_orders.csv


In [4]:
import sqlite3
import pandas as pd

# Load CSV into DataFrame and Some CSVs may need encoding="ISO-8859-1" due to special characters
df = pd.read_csv("raw_orders.csv", encoding="ISO-8859-1")

# Connect to SQLite database (or create one)
conn = sqlite3.connect("ecommerce.db")
cursor = conn.cursor()

# DataFrame to SQL table
df.to_sql("orders", conn, if_exists="replace", index=False)

print("CSV data loaded into SQLite database!")

CSV data loaded into SQLite database!


In [5]:
query = "SELECT * FROM orders LIMIT 5"
result = pd.read_sql_query(query, conn)
print(result)

  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

      InvoiceDate  UnitPrice  CustomerID         Country  
0  12/1/2010 8:26       2.55     17850.0  United Kingdom  
1  12/1/2010 8:26       3.39     17850.0  United Kingdom  
2  12/1/2010 8:26       2.75     17850.0  United Kingdom  
3  12/1/2010 8:26       3.39     17850.0  United Kingdom  
4  12/1/2010 8:26       3.39     17850.0  United Kingdom  


In [6]:
# Storing the SQL query into the Dataframe
cleaned_df = pd.read_sql_query("""
SELECT
    InvoiceNo,
    StockCode,
    TRIM(Description) AS Description,
    Quantity,
    UnitPrice,
    ROUND(Quantity * UnitPrice, 2) AS TotalSales,
    SUBSTR(InvoiceDate, 1, 10) AS OrderDate,
    strftime('%Y-%m', InvoiceDate) AS OrderMonth,
    strftime('%w', InvoiceDate) AS OrderDayOfWeek,
    CustomerID,
    TRIM(Country) AS Country
FROM orders
WHERE
    Quantity > 0 AND
    UnitPrice > 0 AND
    CustomerID IS NOT NULL AND
    LENGTH(InvoiceNo) <= 6 AND
    Description NOT LIKE '%SAMPLE PACK%' AND
    Description NOT LIKE '%DOTCOM%'
ORDER BY InvoiceDate;
""", conn)

# Show first few rows of cleaned data
print("Full data cleaning complete!")
print("\nFirst 5 rows of cleaned data:")
print(cleaned_df.head())

Full data cleaning complete!

First 5 rows of cleaned data:
  InvoiceNo StockCode                       Description  Quantity  UnitPrice  \
0    540561     22343    PARTY PIZZA DISH RED RETROSPOT        24       0.21   
1    540561     22344    PARTY PIZZA DISH PINK POLKADOT        24       0.21   
2    540561    47504H         ENGLISH ROSE SPIRIT LEVEL        24       0.85   
3    540561    84509G     SET OF 4 FAIRY CAKE PLACEMATS        12       1.25   
4    540561     22720  SET OF 3 CAKE TINS PANTRY DESIGN         3       4.95   

   TotalSales   OrderDate OrderMonth OrderDayOfWeek  CustomerID  \
0        5.04  1/10/2011        None           None     13004.0   
1        5.04  1/10/2011        None           None     13004.0   
2       20.40  1/10/2011        None           None     13004.0   
3       15.00  1/10/2011        None           None     13004.0   
4       14.85  1/10/2011        None           None     13004.0   

          Country  
0  United Kingdom  
1  United Kingdo

In [7]:
cleaned_df.to_csv("cleaned_data.csv", index=False)
print("Cleaned dataset saved as cleaned_data.csv")

Cleaned dataset saved as cleaned_data.csv


In [8]:
import os

# Defining the base folder path in Google Drive
drive_base = '/content/drive/MyDrive/ECommerce_Customer_Analytics'

# Creating all required directories as they are not present in my google drive
os.makedirs(os.path.join(drive_base, 'data'), exist_ok=True)
os.makedirs(os.path.join(drive_base, 'sql_queries'), exist_ok=True)
os.makedirs(os.path.join(drive_base, 'reports'), exist_ok=True)

print("Folder structure created in Google Drive")

Folder structure created in Google Drive


In [9]:
# Copying cleaned_data.csv to Google Drive
!cp cleaned_data.csv "/content/drive/MyDrive/ECommerce_Customer_Analytics/data/"

print("Cleaned data file copied to Google Drive")

Cleaned data file copied to Google Drive


In [10]:
import os

# Checking if cleaned_data.csv exists
if os.path.exists("cleaned_data.csv"):
    print("cleaned_data.csv found!")
else:
    print("cleaned_data.csv NOT found.")

cleaned_data.csv found!


In [11]:
import pandas as pd
import sqlite3

# Loading cleaned CSV into DataFrame
cleaned_df = pd.read_csv("cleaned_data.csv")

# Connecting to SQLite DataBase (just in case I missed)
conn = sqlite3.connect("ecommerce.db")

# DataFrame to SQL table
cleaned_df.to_sql("cleaned_data", conn, if_exists="replace", index=False)

print("Table 'cleaned_data' created in SQLite database")

Table 'cleaned_data' created in SQLite database


In [12]:
# Calculating the Recency, Frequency, Monetary Values(RFM Values) for analysing the cutomer's interests
rfm_df = pd.read_sql_query("""
WITH rfm AS (
    SELECT
        CustomerID,
        DATE('now') - MAX(OrderDate) AS Recency,
        COUNT(DISTINCT InvoiceNo) AS Frequency,
        SUM(TotalSales) AS Monetary
    FROM cleaned_data
    GROUP BY CustomerID
)
SELECT
    CustomerID,
    Recency,
    Frequency,
    Monetary
FROM rfm
WHERE CustomerID IS NOT NULL
ORDER BY Monetary DESC;
""", conn)

print("RFM values calculated!")
print("\nFirst 5 rows:")
print(rfm_df.head())

RFM values calculated!

First 5 rows:
   CustomerID  Recency  Frequency   Monetary
0     14646.0     2016         73  280206.02
1     18102.0     2016         60  259657.30
2     17450.0     2016         46  194550.79
3     16446.0     2020          2  168472.50
4     14911.0     2016        201  143825.06


In [13]:
rfm_df.to_csv("rfm_data.csv", index=False)
print("RFM data saved as rfm_data.csv")

RFM data saved as rfm_data.csv


In [14]:
!cp rfm_data.csv "/content/drive/MyDrive/ECommerce_Customer_Analytics/data/"
print("RFM file copied to Google Drive")

RFM file copied to Google Drive


In [15]:
# This is for calculating the RFM Scores based on the percentiles
rfm_scored_df = pd.read_sql_query("""
WITH rfm AS (
    SELECT
        CustomerID,
        DATE('now') - MAX(OrderDate) AS Recency,
        COUNT(DISTINCT InvoiceNo) AS Frequency,
        SUM(TotalSales) AS Monetary
    FROM cleaned_data
    GROUP BY CustomerID
),
rfm_scores AS (
    SELECT
        CustomerID,
        NTILE(5) OVER (ORDER BY Recency DESC) AS R_Score,
        NTILE(5) OVER (ORDER BY Frequency) AS F_Score,
        NTILE(5) OVER (ORDER BY Monetary) AS M_Score,
        NTILE(5) OVER (ORDER BY Recency DESC) ||
        NTILE(5) OVER (ORDER BY Frequency) ||
        NTILE(5) OVER (ORDER BY Monetary) AS RFM_Score
    FROM rfm
)
SELECT
    CustomerID,
    R_Score,
    F_Score,
    M_Score,
    RFM_Score
FROM rfm_scores
ORDER BY RFM_Score DESC;
""", conn)

print("RFM scoring complete!")
print("\nFirst 5 scored customers:")
print(rfm_scored_df.head())

RFM scoring complete!

First 5 scored customers:
   CustomerID  R_Score  F_Score  M_Score RFM_Score
0     17509.0        5        5        5       555
1     17203.0        5        5        5       555
2     18139.0        5        5        5       555
3     12856.0        5        5        5       555
4     15993.0        5        5        5       555


In [16]:
'''
 Adding the Segment labels as:
  Champions
  Loyal Customers
  At Risk
  About to SLeep
  Promising
  Needs Attention
  Others
'''
def get_segment(row):
    r, f, m = row['R_Score'], row['F_Score'], row['M_Score']
    rfm = str(r) + str(f) + str(m)

    if r == 5 and f == 5 and m == 5:
        return 'Champions'
    elif int(rfm) >= 540 or int(rfm) >= 450 or int(rfm) >= 445:
        return 'Loyal Customers'
    elif r == 1 or f == 1 or m == 1:
        return 'At Risk'
    elif r == 5 and (f == 1 or m == 1):
        return 'About to Sleep'
    elif (r == 4 and f == 3) or (r == 4 and f == 2):
        return 'Promising'
    elif r <= 3 and f <= 3 and m <= 3:
        return 'Needs Attention'
    else:
        return 'Other'

# Apply the function
rfm_scored_df['Segment'] = rfm_scored_df.apply(get_segment, axis=1)

print("Customer segments added!")
print("\nFirst 10 customers with segments:")
print(rfm_scored_df.head(10))

Customer segments added!

First 10 customers with segments:
   CustomerID  R_Score  F_Score  M_Score RFM_Score          Segment
0     17509.0        5        5        5       555        Champions
1     17203.0        5        5        5       555        Champions
2     18139.0        5        5        5       555        Champions
3     12856.0        5        5        5       555        Champions
4     15993.0        5        5        5       555        Champions
5     13685.0        5        5        5       555        Champions
6     14547.0        5        5        5       555        Champions
7     13755.0        5        5        5       555        Champions
8     17850.0        5        5        5       555        Champions
9     15473.0        5        5        4       554  Loyal Customers


In [17]:
rfm_scored_df.to_csv("rfm_scored_segmented.csv", index=False)
print("RFM scored & segmented file saved")

RFM scored & segmented file saved


In [18]:
!cp rfm_scored_segmented.csv "/content/drive/MyDrive/ECommerce_Customer_Analytics/data/"
print("File copied to Google Drive")

File copied to Google Drive


In [19]:
# Changing the OrderDate format
conn.execute("""
UPDATE cleaned_data
SET OrderDate = SUBSTR('00' || SUBSTR(OrderDate, INSTR(OrderDate, '/') + 1, 2), -2) || '-' ||
                 SUBSTR('00' || SUBSTR(OrderDate, 1, INSTR(OrderDate, '/') - 1), -2) || '-01'
WHERE OrderDate GLOB '*/*'
""")

print("Fixed MM/DD/YYYY to YYYY-MM-DD format")

Fixed MM/DD/YYYY to YYYY-MM-DD format


In [20]:
fixed_dates = pd.read_sql_query("SELECT DISTINCT OrderDate FROM cleaned_data LIMIT 5", conn)
print("Sample fixed OrderDates:")
print(fixed_dates)

Sample fixed OrderDates:
  OrderDate
0  10-01-01
1  11-01-01
2  12-01-01
3  13-01-01
4  14-01-01


In [21]:
# Recreating a First Purchase table (got an error initially so dropping that table)
conn.execute("DROP TABLE IF EXISTS first_purchase")

conn.execute("""
CREATE TABLE first_purchase AS
SELECT
    CustomerID,
    MIN(OrderDate) AS FirstPurchaseDate
FROM cleaned_data
GROUP BY CustomerID
""")

print("First purchase table recreated with fixed dates")

First purchase table recreated with fixed dates


In [22]:
# Creating a Cohort table
cohort_table = pd.read_sql_query("""
WITH cohort_base AS (
    SELECT
        f.FirstPurchaseDate,
        c.OrderDate,
        strftime('%Y-%m', f.FirstPurchaseDate) AS CohortMonth,
        strftime('%Y-%m', c.OrderDate) AS OrderMonth,
        (strftime('%m', c.OrderDate) - strftime('%m', f.FirstPurchaseDate)) +
        (strftime('%Y', c.OrderDate) - strftime('%Y', f.FirstPurchaseDate)) * 12 AS MonthNumber
    FROM cleaned_data c
    JOIN first_purchase f ON c.CustomerID = f.CustomerID
    WHERE c.CustomerID IS NOT NULL
)
SELECT
    CohortMonth,
    MonthNumber,
    COUNT(DISTINCT CustomerID) AS TotalUsers
FROM cohort_base
GROUP BY CohortMonth, MonthNumber
ORDER BY CohortMonth, MonthNumber;
""", conn)

print("Cohort table rebuilt with fixed dates")
print(cohort_table.head())

DatabaseError: Execution failed on sql '
WITH cohort_base AS (
    SELECT
        f.FirstPurchaseDate,
        c.OrderDate,
        strftime('%Y-%m', f.FirstPurchaseDate) AS CohortMonth,
        strftime('%Y-%m', c.OrderDate) AS OrderMonth,
        (strftime('%m', c.OrderDate) - strftime('%m', f.FirstPurchaseDate)) +
        (strftime('%Y', c.OrderDate) - strftime('%Y', f.FirstPurchaseDate)) * 12 AS MonthNumber
    FROM cleaned_data c
    JOIN first_purchase f ON c.CustomerID = f.CustomerID
    WHERE c.CustomerID IS NOT NULL
)
SELECT
    CohortMonth,
    MonthNumber,
    COUNT(DISTINCT CustomerID) AS TotalUsers
FROM cohort_base
GROUP BY CohortMonth, MonthNumber
ORDER BY CohortMonth, MonthNumber;
': no such column: CustomerID

In [23]:
# Checking the Columns of cleaned_data
columns = pd.read_sql_query("PRAGMA table_info(cleaned_data)", conn)
print("Columns in cleaned_data:")
print(columns[['name', 'type']])

Columns in cleaned_data:
              name     type
0        InvoiceNo  INTEGER
1        StockCode     TEXT
2      Description     TEXT
3         Quantity  INTEGER
4        UnitPrice     REAL
5       TotalSales     REAL
6        OrderDate     TEXT
7       OrderMonth     REAL
8   OrderDayOfWeek     REAL
9       CustomerID     REAL
10         Country     TEXT


In [24]:
# Recreating the First Purchace table
conn.execute("DROP TABLE IF EXISTS first_purchase")

conn.execute("""
CREATE TABLE first_purchase AS
SELECT
    CustomerID,
    MIN(OrderDate) AS FirstPurchaseDate
FROM cleaned_data
GROUP BY CustomerID
""")

print("First purchase table recreated with CustomerID")

First purchase table recreated with CustomerID


In [25]:
test_query = pd.read_sql_query("SELECT CustomerID FROM cleaned_data LIMIT 5", conn)
print("Sample CustomerID values:")
print(test_query)

Sample CustomerID values:
   CustomerID
0     13004.0
1     13004.0
2     13004.0
3     13004.0
4     13004.0


In [26]:
# Creating a Join Between cleaned_data and first_purchase
join_test = pd.read_sql_query("""
SELECT
    c.CustomerID,
    c.OrderDate,
    f.FirstPurchaseDate
FROM cleaned_data c
JOIN first_purchase f ON c.CustomerID = f.CustomerID
LIMIT 2
""", conn)

print("Join result:")
print(join_test)

Join result:
   CustomerID   OrderDate FirstPurchaseDate
0     13004.0  1/10/2011         1/10/2011 
1     13004.0  1/10/2011         1/10/2011 


In [27]:
# Recreating the first purchase table
conn.execute("DROP TABLE IF EXISTS first_purchase")

conn.execute("""
CREATE TABLE first_purchase AS
SELECT
    CustomerID,
    MIN(OrderDate) AS FirstPurchaseDate
FROM cleaned_data
GROUP BY CustomerID
""")

print("first_purchase table recreated")

first_purchase table recreated


In [28]:
# Rebuilding Cohort Base Table
cohort_base = pd.read_sql_query("""
SELECT
    c.CustomerID,
    c.OrderDate,
    f.FirstPurchaseDate,
    strftime('%Y-%m', f.FirstPurchaseDate) AS CohortMonth,
    strftime('%Y-%m', c.OrderDate) AS OrderMonth,
    (strftime('%m', c.OrderDate) - strftime('%m', f.FirstPurchaseDate)) +
    (strftime('%Y', c.OrderDate) - strftime('%Y', f.FirstPurchaseDate)) * 12 AS MonthNumber
FROM cleaned_data c
JOIN first_purchase f ON c.CustomerID = f.CustomerID
WHERE c.CustomerID IS NOT NULL
LIMIT 5
""", conn)

print("Cohort base table:")
print(cohort_base)

Cohort base table:
   CustomerID   OrderDate FirstPurchaseDate CohortMonth OrderMonth MonthNumber
0     13004.0  1/10/2011         1/10/2011         None       None        None
1     13004.0  1/10/2011         1/10/2011         None       None        None
2     13004.0  1/10/2011         1/10/2011         None       None        None
3     13004.0  1/10/2011         1/10/2011         None       None        None
4     13004.0  1/10/2011         1/10/2011         None       None        None


In [29]:
# Fixing date format in cleaned_data
conn.execute("""
UPDATE cleaned_data
SET OrderDate = SUBSTR('00' || SUBSTR(OrderDate, INSTR(OrderDate, '/') + 1, 2), -2) || '-' ||
                 SUBSTR('00' || SUBSTR(OrderDate, 1, INSTR(OrderDate, '/') - 1), -2) || '-01'
WHERE OrderDate GLOB '*/*'
""")

print("Fixed OrderDate format to YYYY-MM-DD")

Fixed OrderDate format to YYYY-MM-DD


In [30]:
# Updating the first purchase
conn.execute("DROP TABLE IF EXISTS first_purchase")

conn.execute("""
CREATE TABLE first_purchase AS
SELECT
    CustomerID,
    MIN(OrderDate) AS FirstPurchaseDate
FROM cleaned_data
GROUP BY CustomerID
""")

print("Updated first_purchase table with fixed dates")

Updated first_purchase table with fixed dates


In [31]:
# Verifying the dates format
fixed_dates = pd.read_sql_query("SELECT DISTINCT OrderDate FROM cleaned_data LIMIT 5", conn)
print("Sample fixed OrderDates:")
print(fixed_dates)

Sample fixed OrderDates:
  OrderDate
0  10-01-01
1  11-01-01
2  12-01-01
3  13-01-01
4  14-01-01


In [32]:
# Year is in 2 digit format. So the cohort table went wrong
conn.execute("""
UPDATE cleaned_data
SET OrderDate =
    CASE WHEN SUBSTR(OrderDate, 1, 2) < '25' THEN '20' || SUBSTR(OrderDate, 1, 2)
         ELSE '19' || SUBSTR(OrderDate, 1, 2)
    END || '-01-01'
""")

print("Fixed 2-digit year to 4-digit year in OrderDate")

Fixed 2-digit year to 4-digit year in OrderDate


In [33]:
fixed_dates = pd.read_sql_query("SELECT DISTINCT OrderDate FROM cleaned_data LIMIT 5", conn)
print("Sample corrected OrderDates:")
print(fixed_dates)

Sample corrected OrderDates:
    OrderDate
0  2010-01-01
1  2011-01-01
2  2012-01-01
3  2013-01-01
4  2014-01-01


In [34]:
# Recreating first purchase table by dropping previous table
conn.execute("DROP TABLE IF EXISTS first_purchase")

conn.execute("""
CREATE TABLE first_purchase AS
SELECT
    CustomerID,
    MIN(OrderDate) AS FirstPurchaseDate
FROM cleaned_data
GROUP BY CustomerID
""")

print("Updated first_purchase table with correct year format")

Updated first_purchase table with correct year format


In [35]:
# Recreating the Cohort table
cohort_base_fixed = pd.read_sql_query("""
WITH cohort_base AS (
    SELECT
        c.CustomerID,
        c.OrderDate,
        f.FirstPurchaseDate,
        strftime('%Y-%m', f.FirstPurchaseDate) AS CohortMonth,
        strftime('%Y-%m', c.OrderDate) AS OrderMonth,
        (strftime('%m', c.OrderDate) - strftime('%m', f.FirstPurchaseDate)) +
        (strftime('%Y', c.OrderDate) - strftime('%Y', f.FirstPurchaseDate)) * 12 AS MonthNumber
    FROM cleaned_data c
    JOIN first_purchase f ON c.CustomerID = f.CustomerID
    WHERE c.CustomerID IS NOT NULL
)
SELECT *
FROM cohort_base
LIMIT 5;
""", conn)

print("Cohort base with real dates:")
print(cohort_base_fixed.head())

Cohort base with real dates:
   CustomerID   OrderDate FirstPurchaseDate CohortMonth OrderMonth  \
0     13004.0  2010-01-01        1927-01-01     1927-01    2010-01   
1     13004.0  2010-01-01        1927-01-01     1927-01    2010-01   
2     13004.0  2010-01-01        1927-01-01     1927-01    2010-01   
3     13004.0  2010-01-01        1927-01-01     1927-01    2010-01   
4     13004.0  2010-01-01        1927-01-01     1927-01    2010-01   

   MonthNumber  
0          996  
1          996  
2          996  
3          996  
4          996  


In [36]:
bad_dates = pd.read_sql_query("""
SELECT CustomerID, OrderDate
FROM cleaned_data
WHERE CustomerID = 13004.0
ORDER BY OrderDate
LIMIT 5
""", conn)

print("Sample OrderDates for Customer 13004:")
print(bad_dates)

Sample OrderDates for Customer 13004:
   CustomerID   OrderDate
0     13004.0  1927-01-01
1     13004.0  1927-01-01
2     13004.0  1927-01-01
3     13004.0  1927-01-01
4     13004.0  1927-01-01


In [37]:
raw_dates = pd.read_sql_query("""
SELECT InvoiceNo, CustomerID, OrderDate
FROM cleaned_data
WHERE CustomerID = 13004.0
LIMIT 5
""", conn)

print("Raw OrderDates for Customer 13004:")
print(raw_dates)

Raw OrderDates for Customer 13004:
   InvoiceNo  CustomerID   OrderDate
0     540561     13004.0  2010-01-01
1     540561     13004.0  2010-01-01
2     540561     13004.0  2010-01-01
3     540561     13004.0  2010-01-01
4     540561     13004.0  2010-01-01


In [38]:
# Removing the bad dates
conn.execute("""
DELETE FROM cleaned_data
WHERE OrderDate < '2000-01-01'
""")

print("Deleted orders with invalid dates (< year 2000)")

Deleted orders with invalid dates (< year 2000)


In [39]:
# Verifing the deletion
valid_dates = pd.read_sql_query("""
SELECT DISTINCT OrderDate
FROM cleaned_data
WHERE CustomerID = 13004.0
LIMIT 5
""", conn)

print("Sample valid OrderDates:")
print(valid_dates)

Sample valid OrderDates:
    OrderDate
0  2010-01-01
1  2013-01-01
2  2017-01-01
3  2015-01-01
4  2016-01-01


In [40]:
# Updating the first Purchase table
conn.execute("DROP TABLE IF EXISTS first_purchase")

conn.execute("""
CREATE TABLE first_purchase AS
SELECT
    CustomerID,
    MIN(OrderDate) AS FirstPurchaseDate
FROM cleaned_data
GROUP BY CustomerID
""")

print("Updated first_purchase table with real dates only")

Updated first_purchase table with real dates only


In [41]:
# Verification
first_purchase_check = pd.read_sql_query("""
SELECT * FROM first_purchase
WHERE CustomerID = 13004.0
""", conn)

print("First purchase date after fix:")
print(first_purchase_check)

First purchase date after fix:
   CustomerID FirstPurchaseDate
0     13004.0        2010-01-01


In [42]:
# Rebuilding Cohort Base Table with Real Dates
cohort_base_fixed = pd.read_sql_query("""
WITH cohort_base AS (
    SELECT
        c.CustomerID,
        c.OrderDate,
        f.FirstPurchaseDate,
        strftime('%Y-%m', f.FirstPurchaseDate) AS CohortMonth,
        strftime('%Y-%m', c.OrderDate) AS OrderMonth,
        (strftime('%m', c.OrderDate) - strftime('%m', f.FirstPurchaseDate)) +
        (strftime('%Y', c.OrderDate) - strftime('%Y', f.FirstPurchaseDate)) * 12 AS MonthNumber
    FROM cleaned_data c
    JOIN first_purchase f ON c.CustomerID = f.CustomerID
    WHERE c.CustomerID IS NOT NULL
)
SELECT *
FROM cohort_base
LIMIT 5;
""", conn)

print("Cohort base with corrected dates:")
print(cohort_base_fixed.head())

Cohort base with corrected dates:
   CustomerID   OrderDate FirstPurchaseDate CohortMonth OrderMonth  \
0     13004.0  2010-01-01        2010-01-01     2010-01    2010-01   
1     13004.0  2010-01-01        2010-01-01     2010-01    2010-01   
2     13004.0  2010-01-01        2010-01-01     2010-01    2010-01   
3     13004.0  2010-01-01        2010-01-01     2010-01    2010-01   
4     13004.0  2010-01-01        2010-01-01     2010-01    2010-01   

   MonthNumber  
0            0  
1            0  
2            0  
3            0  
4            0  


In [43]:
# Building Final Cohort Table Again
cohort_final_fixed = pd.read_sql_query("""
SELECT
    CohortMonth,
    MonthNumber,
    COUNT(DISTINCT CustomerID) AS TotalUsers
FROM (
    SELECT
        c.CustomerID,
        strftime('%Y-%m', f.FirstPurchaseDate) AS CohortMonth,
        (strftime('%m', c.OrderDate) - strftime('%m', f.FirstPurchaseDate)) +
        (strftime('%Y', c.OrderDate) - strftime('%Y', f.FirstPurchaseDate)) * 12 AS MonthNumber
    FROM cleaned_data c
    JOIN first_purchase f ON c.CustomerID = f.CustomerID
    WHERE c.CustomerID IS NOT NULL
)
GROUP BY CohortMonth, MonthNumber
ORDER BY CohortMonth, MonthNumber
LIMIT 10;
""", conn)

print("Final cohort table with fixed dates:")
print(cohort_final_fixed.head(10))

Final cohort table with fixed dates:
  CohortMonth  MonthNumber  TotalUsers
0        None          NaN         673
1     2010-01          NaN          47
2     2010-01          0.0         383
3     2010-01         12.0          59
4     2010-01         24.0          41
5     2010-01         36.0          54
6     2010-01         48.0          46
7     2010-01         60.0          54
8     2010-01         72.0          57
9     2010-01         84.0          52


In [44]:
# Pivoting the data for visualisation which shows how many users retained
cohort_pivot_fixed = cohort_final_fixed.pivot_table(
    index='CohortMonth',
    columns='MonthNumber',
    values='TotalUsers'
)

# Optional: Convert to percentages
cohort_retention_fixed = cohort_pivot_fixed.divide(cohort_pivot_fixed.iloc[:, 0], axis=0) * 100

print("Cohort pivoted for visualization")
print("\nRetention Table (%):")
print(cohort_retention_fixed.round(1).head())

Cohort pivoted for visualization

Retention Table (%):
MonthNumber   0.0   12.0  24.0  36.0  48.0  60.0  72.0  84.0
CohortMonth                                                 
2010-01      100.0  15.4  10.7  14.1  12.0  14.1  14.9  13.6


In [45]:
cohort_pivot_fixed.to_csv("cohort_analysis.csv")
cohort_retention_fixed.to_csv("cohort_retention_percent.csv")

print("Cohort tables saved to CSV")

Cohort tables saved to CSV


In [46]:
!cp cohort_analysis.csv "/content/drive/MyDrive/ECommerce_Customer_Analytics/data/"
!cp cohort_retention_percent.csv "/content/drive/MyDrive/ECommerce_Customer_Analytics/data/"

print("Files copied to Google Drive")

Files copied to Google Drive


In [47]:
#  A/B Test Simulation (Randomly assigning customers to either groupA(got promo) or groupB(no promo))
ab_test_df = pd.read_sql_query("""
SELECT
    CustomerID,
    CASE WHEN RANDOM() % 2 = 0 THEN 'A' ELSE 'B' END AS GroupLabel
FROM first_purchase
GROUP BY CustomerID
""", conn)

print("Assigned customers to Group A or B")
print(ab_test_df.head())

Assigned customers to Group A or B
   CustomerID GroupLabel
0     12346.0          B
1     12347.0          A
2     12348.0          B
3     12349.0          A
4     12350.0          A


In [48]:
ab_test_df.to_sql("ab_groups", conn, if_exists="replace", index=False)
print("Created ab_groups table")

Created ab_groups table


In [49]:
# Calculating the total spending per customer and joining with group
spending_by_group = pd.read_sql_query("""
SELECT
    g.GroupLabel,
    SUM(c.TotalSales) AS TotalSpent
FROM cleaned_data c
JOIN ab_groups g ON c.CustomerID = g.CustomerID
GROUP BY g.GroupLabel
""", conn)

print("Spending by group:")
print(spending_by_group)

Spending by group:
  GroupLabel  TotalSpent
0          A  2410342.81
1          B  2600506.95


In [50]:
# Calculating Average Spend Per User
avg_spending = pd.read_sql_query("""
SELECT
    g.GroupLabel,
    AVG(c.TotalSales) AS AvgSpentPerUser
FROM (
    SELECT CustomerID, SUM(TotalSales) AS TotalSales
    FROM cleaned_data
    GROUP BY CustomerID
) c
JOIN ab_groups g ON c.CustomerID = g.CustomerID
GROUP BY g.GroupLabel
""", conn)

print("Average spending per user by group:")
print(avg_spending)

Average spending per user by group:
  GroupLabel  AvgSpentPerUser
0          A      1412.034452
1          B      1512.802182


In [51]:
# Statistical Test (T-Test)
from scipy import stats

# Get individual user spending
user_spending = pd.read_sql_query("""
SELECT
    g.GroupLabel,
    SUM(c.TotalSales) AS TotalSpent
FROM cleaned_data c
JOIN ab_groups g ON c.CustomerID = g.CustomerID
GROUP BY c.CustomerID, g.GroupLabel
""", conn)

# Separate into Group A and B
group_a = user_spending[user_spending['GroupLabel'] == 'A']['TotalSpent']
group_b = user_spending[user_spending['GroupLabel'] == 'B']['TotalSpent']

# T-test
t_stat, p_value = stats.ttest_ind(group_a, group_b, equal_var=False)

print(f"T-statistic: {t_stat:.3f}")
print(f"P-value: {p_value:.3f}")

T-statistic: -0.490
P-value: 0.624


In [52]:
user_spending.to_csv("ab_user_spending.csv", index=False)
avg_spending.to_csv("ab_avg_spending.csv", index=False)

print("A/B test files saved")

A/B test files saved


In [53]:
!cp ab_user_spending.csv "/content/drive/MyDrive/ECommerce_Customer_Analytics/data/"
!cp ab_avg_spending.csv "/content/drive/MyDrive/ECommerce_Customer_Analytics/data/"

print("Files copied to Google Drive")

Files copied to Google Drive
